In [79]:
import rasa
import numpy as np
from pathlib import Path
import pandas as pd
from rasa.nlu.model import Interpreter
from rasa.cli.utils import get_validated_path
from rasa.constants import DEFAULT_MODELS_PATH
from rasa.model import get_model, get_model_subdirectories

In [28]:
!pwd

/d/dev/sunmi/git/chatbot_nlu/cs/tests


In [29]:
parent_folder = Path('.').absolute().parents[0]

In [34]:
model = str(parent_folder / "models")
model = get_validated_path(model, "model", DEFAULT_MODELS_PATH)
model_path = get_model(model)
_, nlu_model = get_model_subdirectories(model_path)
interpreter = Interpreter.load(nlu_model)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\wegam\AppData\Local\Temp\jieba.cache
Loading model cost 0.751 seconds.
Prefix dict has been built succesfully.


server config:
                        client	=	ffcbb6e1-5370-446e-9500-cda4baae0209
                   num_process	=	2                             
          ventilator -> worker	=	['ipc://tmpxU77SW/socket', 'ipc://tmprAoqej/socket', 'ipc://tmpDlRKzF/socket', 'ipc://tmprjr7U1/socket', 'ipc://tmpvFewgo/socket', 'ipc://tmp761XBK/socket', 'ipc://tmpZLXrX6/socket', 'ipc://tmpkznYit/socket']
                worker -> sink	=	ipc://tmpRvQjdr/socket        
           ventilator <-> sink	=	ipc://tmpM9bSxA/socket        
           server_current_time	=	2019-08-29 13:48:40.603695    
                    device_map	=	[]                            
         num_concurrent_socket	=	8                             
                     ckpt_name	=	bert_model.ckpt               
                   config_name	=	bert_config.json              
                          cors	=	*                             
                           cpu	=	False                         
                 do_lower_case	=	

                server_version	=	1.9.6                         
                 pyzmq_version	=	18.0.2                        
                   zmq_version	=	4.3.1                         
             server_start_time	=	2019-08-29 11:04:33.799497    
server config:
                        client	=	f09b507e-f199-4c8d-89d6-86219818aa78
                   num_process	=	2                             
          ventilator -> worker	=	['ipc://tmpxU77SW/socket', 'ipc://tmprAoqej/socket', 'ipc://tmpDlRKzF/socket', 'ipc://tmprjr7U1/socket', 'ipc://tmpvFewgo/socket', 'ipc://tmp761XBK/socket', 'ipc://tmpZLXrX6/socket', 'ipc://tmpkznYit/socket']
                worker -> sink	=	ipc://tmpRvQjdr/socket        
           ventilator <-> sink	=	ipc://tmpM9bSxA/socket        
           server_current_time	=	2019-08-29 13:48:41.181073    
                    device_map	=	[]                            
         num_concurrent_socket	=	8                             
                     ckpt_name	=	

                           xla	=	False                         
            tensorflow_version	=	['1', '13', '1']              
                python_version	=	3.6.9 |Anaconda, Inc.| (default, Jul 30 2019, 19:07:31) 
[GCC 7.3.0]
                server_version	=	1.9.6                         
                 pyzmq_version	=	18.0.2                        
                   zmq_version	=	4.3.1                         
             server_start_time	=	2019-08-29 11:04:33.799497    
server config:
                        client	=	4454bdb2-55bb-4ec0-9136-2b3df3da8bbd
                   num_process	=	2                             
          ventilator -> worker	=	['ipc://tmpxU77SW/socket', 'ipc://tmprAoqej/socket', 'ipc://tmpDlRKzF/socket', 'ipc://tmprjr7U1/socket', 'ipc://tmpvFewgo/socket', 'ipc://tmp761XBK/socket', 'ipc://tmpZLXrX6/socket', 'ipc://tmpkznYit/socket']
                worker -> sink	=	ipc://tmpRvQjdr/socket        
           ventilator <-> sink	=	ipc://tmpM9bSxA/socket    

           priority_batch_size	=	16                            
         show_tokens_to_client	=	False                         
               tuned_model_dir	=	None                          
                       verbose	=	False                         
                           xla	=	False                         
            tensorflow_version	=	['1', '13', '1']              
                python_version	=	3.6.9 |Anaconda, Inc.| (default, Jul 30 2019, 19:07:31) 
[GCC 7.3.0]
                server_version	=	1.9.6                         
                 pyzmq_version	=	18.0.2                        
                   zmq_version	=	4.3.1                         
             server_start_time	=	2019-08-29 11:04:33.799497    
server config:
                        client	=	b3142b92-81cb-4fa7-b986-815211bbb384
                   num_process	=	2                             
          ventilator -> worker	=	['ipc://tmpxU77SW/socket', 'ipc://tmprAoqej/socket', 'ipc://tmpDlRKzF/socket

## 1. Raw Data Statistics

In [39]:
df = pd.read_csv('chats.csv', index_col=0)

In [62]:
num_services = len(df.from_user_name.apply(find_kf).unique()) - 1
num_groups = len(df.chat_group_wxid.unique())
num_customers = len(df.from_user_wxid.unique()) - services
num_chats = len(df)

print('Number of services: {0}'.format(num_services))
print('Number of Groups: {0}'.format(num_groups))
print('Number of Users: {0}'.format(num_customers))
print('Number of Users: {0}'.format(num_chats))

Number of services: 8
Number of Groups: 177
Number of Users: 1128
Number of Users: 25109


## 2. Filter out Useful Questions

In [82]:
parsed_result = df.chat_text.apply(lambda x: interpreter.parse(x))

D:\ProgramData\Anaconda3\envs\rasa\lib\site-packages\bert_serving\client\__init__.py:299: UserWarning: some of your sentences have more tokens than "max_seq_len=50" set on the server, as consequence you may get less-accurate or truncated embeddings.
here is what you can do:
- disable the length-check by create a new "BertClient(check_length=False)" when you do not want to display this warning
- or, start a new server with a larger "max_seq_len"
  '- or, start a new server with a larger "max_seq_len"' % self.length_limit)


In [96]:
is_question = [True if r['intent']['name'] == 'question' else False for r in parsed_result]
df['is_question'] = is_question

In [94]:
print("Similar as question: {0}".format(np.sum(is_question)))

Similar as question: 15241
